<a href="https://colab.research.google.com/github/richmondvan/ringview/blob/master/RingviewTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Imports

In [ ]:
from google.colab.patches import cv2_imshow as imshow # For image previews
import numpy as np # for linear algebra help
from math import inf, log2 # for math help :D
from random import randint, random # for random number generation
from skimage.metrics import structural_similarity, mean_squared_error # For similarity measurement
import cv2 # opencv2 for image management

from PIL import Image, ImageFilter # for initial image palette and bgcolor generation with help of colorthief
%pip install colorthief
from colorthief import ColorThief # For grabbing color information from source image

### Create some helper methods

In [ ]:
def changeColorFromRGBToBGR(color): # converts from RGB (colorthief colors) to BGR format (opencv2 colors)
    return (color[2], color[1], color[0])

def showImage(image, adjuster, adjustment=False): # Shows the image, this is placeholder for VSCode to Colab conversions (since cv2_imshow does not work outside of colab)
    if adjustment:
        restoredImage = adjuster.unadjust(image)
    else:
        restoredImage = image
    imshow(restoredImage)

class MeanSquaredError:
    def calculateError(self, image, source):
        return mean_squared_error(image, source)

class StructuralSimilarityError:
    def calculateError(self, image, source):
        return -1 * structural_similarity(image, source, multichannel=True)

errorChecker = MeanSquaredError()

def addCircle(image, gene): # Adds a circle based on gene information onto an image using opencv2 blending modes.
    overlay = image.copy()
    cv2.circle(
        overlay,
        center=gene.center,
        radius=gene.radius,
        color=palette[gene.color],
        thickness=-1,
        lineType=cv2.LINE_AA # add antialiasing
    )
    cv2.addWeighted(overlay, gene.alpha, image, 1-gene.alpha, 0, image)

def getRescaleRatio(image):
    # Target: 250x250
    width = image.width
    height = image.height

    if width <= 250 or height <= 250:
        return 1

    ratio = 250 / min(width, height)
    return ratio

class NoAdjuster():
    def adjust(self, originalImage):
        return originalImage

    def unadjust(self, originalImage):
        return originalImage

class DarkAdjuster():
    def __init__():
        self.ADJUST_A = 17.1976
        self.ADJUST_B = 256
        self.ADJUST_C = 512
        self.ADJUST_D = -262.665
    

    def adjust(self, originalImage):
        # image is a numpy array
        adjustedImage = originalImage.copy()
        adjustedImage = adjustedImage + self.ADJUST_A
        adjustedImage = np.log(adjustedImage) / np.log(self.ADJUST_B)
        adjustedImage = adjustedImage * self.ADJUST_C
        adjustedImage = adjustedImage + self.ADJUST_D
        adjustedImage = np.rint(adjustedImage).astype(np.uint8)
        return adjustedImage


    def unadjust(self, adjustedImage):
        # image is a numpy array
        originalImage = adjustedImage.copy()
        originalImage = originalImage - self.ADJUST_D
        originalImage = originalImage / self.ADJUST_C
        originalImage = np.exp2(log2(self.ADJUST_B) * originalImage)
        originalImage = originalImage - self.ADJUST_A
        originalImage = np.rint(originalImage).astype(np.uint8)

        return originalImage

adjuster = NoAdjuster()

class ColorThiefFromImage(ColorThief): # Extend ColorThief to support providing images as-is instead of filenames
    def __init__(self, image):
        self.image = image

### Load an example into memory

In [ ]:
# Trial filenames
BUTTERFLY_FILENAME = "img.jpg"
MONALISA_FILENAME = "monalisa224.png"
MONALISASQUARE_FILENAME = "monalisasquare.png" # Use this!
PEARLEARRING = "pearlearring.jpg"
STARRYNIGHT = "starrynight.png"

NUM_COLORS = 256 # Number of colors in color palette (this is approximate, colorthief is inconsistent)

BASEIMAGE_FILENAME = PEARLEARRING # Set this to filename of proper image

# Smooth image three times
baseImage = Image.open(BASEIMAGE_FILENAME) 
smoothenedImage = baseImage.filter(ImageFilter.SMOOTH_MORE)
smoothenedImage = smoothenedImage.filter(ImageFilter.SMOOTH_MORE)
smoothenedImage = smoothenedImage.filter(ImageFilter.SMOOTH_MORE)

ratio = getRescaleRatio(smoothenedImage)

smoothenedImage = smoothenedImage.resize((int(smoothenedImage.width * ratio), int(smoothenedImage.height * ratio)), Image.LANCZOS)
imageArray = np.asarray(smoothenedImage)

originalBaseImage = cv2.cvtColor(imageArray, cv2.COLOR_RGB2BGR)

imageArray = adjuster.adjust(imageArray)
smoothenedImage = Image.fromarray(imageArray)

BGRbaseImage = cv2.cvtColor(imageArray, cv2.COLOR_RGB2BGR)

imageArray = adjuster.unadjust(imageArray)
restoredBaseImage = cv2.cvtColor(imageArray, cv2.COLOR_RGB2BGR)

# generate some colours out of smoothened image, remember to convert from RGB to BGR
colorThief = ColorThiefFromImage(smoothenedImage)
backgroundColor = changeColorFromRGBToBGR(colorThief.get_color(quality=1)) # gets the background color
palette = colorThief.get_palette(color_count=NUM_COLORS, quality=1) # gets the top colors used by the image.
for index, color in enumerate(palette):
    palette[index] = changeColorFromRGBToBGR(color)

print("Original: ")
showImage(originalBaseImage, adjuster)

print("Adjusted: ")
showImage(BGRbaseImage, adjuster) # Preview image

print("Restored: ")
showImage(restoredBaseImage, adjuster)

### Prepare for evolution

In [ ]:
# Set evolution constants

NUM_GENERATIONS = 50000 # 20000 default. Number of generations to run.

NUM_CIRCLES = 1000 # 1000 default. Number of circles to draw. 

print("Shape of image: ")
print(BGRbaseImage.shape)

# Calculations for minimum and maximum radius to draw on images.
MINIMUM_RADIUS = int(0.02 * min(BGRbaseImage.shape[:1]))
MAXIMUM_RADIUS = int(0.08 * min(BGRbaseImage.shape[:1]))

print("Minimum Circle Radius: ")
print(MINIMUM_RADIUS)

print("Maximum Circle Radius: ")
print(MAXIMUM_RADIUS)

# width and height variables for convenience.
height = BGRbaseImage.shape[0]
width = BGRbaseImage.shape[1]

# Build a base image with a solid background color.
ancestorImage = np.zeros((height, width, 3), np.uint8)
ancestorImage[:] = backgroundColor

print("Background color: ")
showImage(ancestorImage, adjuster) # Preview base image

### Create gene class

In [ ]:
class Gene: # Defines a single gene, which describes how to draw a single circle.
    def __init__(self, maximumRadius, minimumRadius, height, width, numColors, radius=None, center=None, color=None, alpha=None): # Basic constructor
        self.maximumRadius = maximumRadius
        self.minimumRadius = minimumRadius
        self.height = height
        self.width = width
        self.numColors = numColors
        if radius is None:
            self.completelyRandomize() # Randomize values at initialization.
        else:
            self.radius = radius
            self.center = center
            self.color = color
            self.alpha = alpha
    
    def revert(self): # Reverts circle to previous properties (in case a mutation was unhelpful.)
        self.radius = self.history[0]
        self.center = self.history[1]
        self.color = self.history[2]
        self.alpha = self.history[3]
    
    def completelyRandomize(self): # Completely randomizes the values 
        self.radius = randint(self.minimumRadius, self.maximumRadius)
        self.center = (randint(0 - int(self.radius/5), self.width + int(self.radius/5)), randint(0 - int(self.radius/5), self.height + int(self.radius/5)))
        self.color = randint(0, self.numColors-2)
        self.alpha = random() * 0.45 + 0.05

    def mutate(self, complete=False): # Mutates a circle randomly.  If complete is True, this will always perform a complete mutation.
        self.history = [self.radius, self.center, self.color, self.alpha]
        seed = random()
        if seed < 0.5: # 50% chance of complete mutation, 50% chance of just changing the radius of the circle
            self.completelyRandomize()
        else:
            self.radius = randint(self.minimumRadius, self.maximumRadius)
    
    def getScaledVersion(self, ratio):
        newRadius = int(self.radius / ratio)
        newCenter = (int(self.center[0] / ratio), int(self.center[1] / ratio))
        newHeight = int(self.height / ratio)
        newWidth = int(self.width / ratio)
        newMaxRadius = int(self.maximumRadius / ratio)
        newMinRadius = int(self.minimumRadius / ratio)


        newGene = Gene(newMaxRadius, newMinRadius, newHeight, newWidth, self.numColors, newRadius, newCenter, self.color, self.alpha)
        return newGene

### Evolve


In [ ]:
topScore = inf # original best score is infinite, so the first run will always overwrite it.
topImage = None # the best image so far.

cachedImage = None
recallFromCache = False

sequence = [Gene(MAXIMUM_RADIUS, MINIMUM_RADIUS, height, width, NUM_COLORS) for _ in range(NUM_CIRCLES)] # Generate initial gene sequence.

changes = 0 # Record number of total changes made. (some generations may have failed mutations that do not affect the sequence.)

EARLY_STOPPING_LIMIT = 500
generationsSinceLastChange = 0

totalGenerationChanges = 0
consecutiveExtremeMutations = 0
EARLY_STOPPING_LIMIT_EXTREME_MUTATION = 1000
extremeMutation = False

finalRun = False

for generationIndex in range(NUM_GENERATIONS): # loop through each generation

    if extremeMutation:
        finalRun = False

        # Choose a random gene to mutate (not just the bottom one this time.)
        mutationIndex = randint(0, NUM_CIRCLES - 1)
        mutatedGene = sequence[mutationIndex]
        mutatedGene.mutate()

        seed = random()

        if seed > 0.5:
            del sequence[mutationIndex]
            sequence.append(mutatedGene)

        # Redraw image
        image = ancestorImage.copy()
        for gene in sequence:
            addCircle(image, gene)


        # evaluate the mutation
        score = errorChecker.calculateError(image, BGRbaseImage)

        # If successful...
        if score < topScore:
            topScore = score
            topImage = image
            changes += 1
            # Reset all values and go back to regular mutations.
            generationsSinceLastChange = 0
            consecutiveExtremeMutations += 1
            if (consecutiveExtremeMutations >= totalGenerationChanges ** 2):
                recallFromCache = False
                extremeMutation = False
        else:
            generationsSinceLastChange += 1
            mutatedGene.revert()

            if seed > 0.5:
                del sequence[-1]
                sequence.insert(mutationIndex, mutatedGene)

            # If we really can't get anywhere, then quit.
            if generationsSinceLastChange >= EARLY_STOPPING_LIMIT_EXTREME_MUTATION:
                finalRun = True
                generationsSinceLastChange = 0
                recallFromCache = False
                extremeMutation = False
                print("early stop!")
                break

    else:
        # grab the gene at the bottom of the sequence (which corresponds to the circle at the very bottom of the image, depthwise)
        mutatedGene = sequence[0]
        # mutate it
        mutatedGene.mutate()

        # delete the gene
        del sequence[0]

        # add the rest of the circles normally
        if recallFromCache:
            image = cachedImage.copy() # Take built image from 
        else:
            image = ancestorImage.copy()
            for gene in sequence:
                addCircle(image, gene)
            
            cachedImage = image.copy()
            recallFromCache = True

        # add the gene on top.
        addCircle(image, mutatedGene)

        # evaluate the mutation
        score = errorChecker.calculateError(image, BGRbaseImage)

        # if it was beneficial...
        if score < topScore:
            topScore = score
            topImage = image
            sequence.append(mutatedGene) # Place the gene on top of the sequence again.
            changes += 1 # record a change
            generationsSinceLastChange = 0
            recallFromCache = False # Build next image from scratch.
            finalRun = False
        else:
            generationsSinceLastChange += 1
            mutatedGene.revert() # oopsies! revert changes.
            sequence.insert(0, mutatedGene) # insert gene back into the end of the sequence so it can be mutated again next generation.
            if generationsSinceLastChange >= EARLY_STOPPING_LIMIT:
                totalGenerationChanges += 1
                extremeMutation = True
                generationsSinceLastChange = 0
                consecutiveExtremeMutations = 0
                if finalRun:
                    break

    # Periodic checks on progress, every 100 generations.
    if generationIndex % 200 == 0:
        print(f"generation: {generationIndex}")
        print(f"number of changes: {changes}")
        print(f"number of generation changes: {totalGenerationChanges}")
        extreme = "extreme" if extremeMutation else "not extreme"
        print(f"Extreme?: {extreme}")
        showImage(topImage, adjuster, adjustment=True)

showImage(topImage, adjuster, adjustment=True)
print(changes)

In [ ]:
def renderScaledImage(sequence, ratio, height, width, backgroundColor, adjuster):

    scaledHeight = int(height / ratio)
    scaledWidth = int(width / ratio)

    renderedImage = np.zeros((scaledHeight, scaledWidth, 3), np.uint8)
    renderedImage[:] = backgroundColor

    scaledSequence = [gene.getScaledVersion(ratio) for gene in sequence]
    for gene in scaledSequence:
        addCircle(renderedImage, gene)

    showImage(renderedImage, adjuster, adjustment=True)

renderScaledImage(sequence, ratio, height, width, backgroundColor, adjuster)
print(changes)

print(len(sequence))
